In [1]:
from datasets import load_dataset, Dataset
from tqdm import tqdm
from utils import get_response_openai_prompt
import yaml
import pandas as pd
import os
import random
import secrets
import string

In [2]:
def generate_unique_strings(count, length):
    unique_strings = set()
    characters = string.ascii_letters + string.digits
    while len(unique_strings) < count:
        random_string = "".join(secrets.choice(characters) for _ in range(length))
        unique_strings.add(random_string)
    return list(unique_strings)

In [3]:
def load_config(config_file):
    with open(config_file, "r") as file:
        config = yaml.safe_load(file)
    return config

config_path = os.path.join(os.getcwd(), "..", "config.yaml")
with open(config_path, "r") as file:
    config = yaml.safe_load(file)

hf_token = config["huggingface"]["token"]

In [4]:
ds = load_dataset("O1-OPEN/OpenO1-SFT", token=hf_token)
df = Dataset.to_pandas(ds["train"])
df.head()

README.md:   0%|          | 0.00/10.1k [00:00<?, ?B/s]

c:\Users\86183\anaconda3\lib\site-packages\huggingface_hub\file_download.py:147: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\86183\.cache\huggingface\hub\datasets--O1-OPEN--OpenO1-SFT. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


OpenO1-SFT.jsonl:   0%|          | 0.00/384M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/77685 [00:00<?, ? examples/s]

,instruction,output
0,Consider a regular octagon. How many different...,"<Thought>\nAlright, I need to figure out how m..."
1,Create a Python class that encodes a given num...,"<Thought>\nAlright, I need to create a Python ..."
2,Write a Python function named `is_valid_binary...,"<Thought>\nAlright, I need to write a Python f..."
3,Let X be a subset of a Banach space Y with the...,A nice result in functional analysis!\n\nThe s...
4,Identify the correct option for document fetch...,"<Thought>\nAlright, the task at hand is to ide..."


In [5]:
df = df.sample(3000, random_state=3407)
df.reset_index(drop=True, inplace=True)
print(df.shape)
df.head()

(3000, 2)


,instruction,output
0,"Using the jQuery library, write a snippet to a...","<Thought>\nAlright, I need to write a jQuery s..."
1,Suppose your friend has entered a chess tourna...,Here is a MATLAB code that calculates the prob...
2,"On a recent day in Oregon, at 9 a.m. the tempe...","<Thought>\nAlright, let's dive into this probl..."
3,Derivation on Hom Functor for Commutative Diag...,A classic result in homological algebra!\n\nLe...
4,I am making a script for automated tests. But ...,"<Thought>\nAlright, I need to help the user cr..."


In [ ]:
new_data = pd.DataFrame(columns=["prompt", "response_a", "response_b", "winner", "language"])

In [ ]:
for idx, row in tqdm(df.iterrows(), total=len(df)):
    if idx < 1000:
        instruction = row["instruction"]
        translated_instruction = get_response_openai_prompt(f"Translate the following instruction to Russian: {instruction}.\nYou should only return the translation.")